In [2]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [3]:
batch_size = 64
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='./data/',
                             train=False,
                             transform=transforms.ToTensor())
# Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Processing...
Done!


In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 520)
        self.l2 = torch.nn.Linear(520, 320)
        self.l3 = torch.nn.Linear(320, 240)
        self.l4 = torch.nn.Linear(240, 120)
        self.l5 = torch.nn.Linear(120, 10)
    
    def forward(self, x):
        # Flatten (n, 1, 28, 28) -> (n, 784)
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [6]:
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [9]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        output = model(data)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        
        test_loss += criterion(output, target).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()

Train epoch: 1 [0/60000 (0%)]	Loss: 0.032484
Train epoch: 1 [6400/60000 (11%)]	Loss: 0.138866
Train epoch: 1 [12800/60000 (21%)]	Loss: 0.062423
Train epoch: 1 [19200/60000 (32%)]	Loss: 0.169080
Train epoch: 1 [25600/60000 (43%)]	Loss: 0.073476
Train epoch: 1 [32000/60000 (53%)]	Loss: 0.063126
Train epoch: 1 [38400/60000 (64%)]	Loss: 0.024421
Train epoch: 1 [44800/60000 (75%)]	Loss: 0.118166
Train epoch: 1 [51200/60000 (85%)]	Loss: 0.029787
Train epoch: 1 [57600/60000 (96%)]	Loss: 0.098405

Test set: Average loss: 0.0014, Accuracy: 9745/10000 (97%)

Train epoch: 2 [0/60000 (0%)]	Loss: 0.088873
Train epoch: 2 [6400/60000 (11%)]	Loss: 0.028029
Train epoch: 2 [12800/60000 (21%)]	Loss: 0.083557
Train epoch: 2 [19200/60000 (32%)]	Loss: 0.019409
Train epoch: 2 [25600/60000 (43%)]	Loss: 0.014144
Train epoch: 2 [32000/60000 (53%)]	Loss: 0.012644
Train epoch: 2 [38400/60000 (64%)]	Loss: 0.092228
Train epoch: 2 [44800/60000 (75%)]	Loss: 0.064886
Train epoch: 2 [51200/60000 (85%)]	Loss: 0.075325
T